### Install Requirements

In [2]:
!pip install torch torchvision albumentations segmentation-models-pytorch

In [3]:
!pip install scikit-learn

### Download Data

In [4]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("aryashah2k/breast-ultrasound-images-dataset")

print("Path to dataset files:", path)

/Users/sofiautoft/classes/DS/liver_segmentation/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/sofiautoft/.cache/kagglehub/datasets/aryashah2k/breast-ultrasound-images-dataset/versions/1


### Load & Resize Data

In [ ]:
import cv2 as cv
import numpy as np
import os

def load_images_from_folder(dataset_path, img_size=(256, 256)):
    images, masks = [], []
    categories = ['benign', 'normal', 'malignant']

    for category in categories:
        category_path = os.path.join(dataset_path, category)
        image_files = sorted([f for f in os.listdir(category_path) if "mask" not in f.lower()])
        mask_files = sorted([f for f in os.listdir(category_path) if "mask" in f.lower()])

        for img_file, mask_file in zip(image_files, mask_files):
            print(f"Loading: {img_file} and {mask_file}")
            img_path, mask_path = os.path.join(category_path, img_file), os.path.join(category_path, mask_file)
            img, mask = cv.imread(img_path), cv.imread(mask_path, cv.IMREAD_GRAYSCALE)

            if img is not None and mask is not None:
                images.append(cv.resize(img, img_size) / 255.0)
                masks.append(cv.resize(mask, img_size) / 255.0)

    return np.array(images), np.expand_dims(np.array(masks), axis=-1)

data_path = os.path.expanduser("/Users/sofiautoft/.cache/kagglehub/datasets/aryashah2k/breast-ultrasound-images-dataset/versions/1/Dataset_BUSI_with_GT")
images, masks = load_images_from_folder(data_path)


Loading: benign (1).png and benign (1)_mask.png
Loading: benign (10).png and benign (10)_mask.png
Loading: benign (100).png and benign (100)_mask.png
Loading: benign (101).png and benign (100)_mask_1.png
Loading: benign (102).png and benign (101)_mask.png
Loading: benign (103).png and benign (102)_mask.png
Loading: benign (104).png and benign (103)_mask.png
Loading: benign (105).png and benign (104)_mask.png
Loading: benign (106).png and benign (105)_mask.png
Loading: benign (107).png and benign (106)_mask.png
Loading: benign (108).png and benign (107)_mask.png
Loading: benign (109).png and benign (108)_mask.png
Loading: benign (11).png and benign (109)_mask.png
Loading: benign (110).png and benign (11)_mask.png
Loading: benign (111).png and benign (110)_mask.png
Loading: benign (112).png and benign (111)_mask.png
Loading: benign (113).png and benign (112)_mask.png
Loading: benign (114).png and benign (113)_mask.png
Loading: benign (115).png and benign (114)_mask.png
Loading: benign (1

In [ ]:
import segmentation_models_pytorch as smp
import torch
import torch.nn as nn

# Create the DeepLabV3+ model
model = smp.DeepLabV3Plus(
    encoder_name="resnet34",        # or 'resnet50' if you want deeper
    encoder_weights="imagenet",     # use ImageNet pre-trained weights
    in_channels=3,                  # ultrasound images: 3 if RGB, 1 if grayscale
    classes=1                       # binary segmentation (tumor vs. background)
)

In [ ]:
import torch.nn.functional as F

def pad_to_multiple_of_16(image):
    # Get the current height and width of the image
    height, width = image.shape[:2]

    # Compute the padding needed
    pad_h = (16 - height % 16) % 16  # Ensure the pad is a multiple of 16
    pad_w = (16 - width % 16) % 16   # Ensure the pad is a multiple of 16

    # Apply padding to the image
    padded_image = F.pad(image, (0, pad_w, 0, pad_h), mode='constant', value=0)
    return padded_image

In [ ]:
from torch.utils.data import Dataset
import cv2
import numpy as np

class UltrasoundDataset(Dataset):
    def __init__(self, image_arrays, mask_arrays, transforms=None):
        self.image_arrays = image_arrays
        self.mask_arrays = mask_arrays
        self.transforms = transforms

    def __len__(self):
        return len(self.image_arrays)
    
    def __getitem__(self, idx):
        image = self.image_arrays[idx]   # already numpy array
        mask = self.mask_arrays[idx]     # already numpy array
        mylist = [image, mask]
    
        for i in range(2):
            item = mylist[i]
            h, w = item.shape[:2]

            # Compute the padding needed
            new_h = (h + 15) // 16 * 16  # Next multiple of 16
            new_w = (w + 15) // 16 * 16  # Next multiple of 16

            # Apply padding to the image or mask
            mylist[i] = np.pad(item, ((0, new_h - h), (0, new_w - w), (0, 0)), mode='constant', constant_values=0)

        # Convert to torch tensors
        image = torch.tensor(mylist[0]).permute(2, 0, 1).float()
        mask = torch.tensor(mylist[1]).permute(2, 0, 1).float()

        return image, mask

In [ ]:
loss_fn = nn.BCEWithLogitsLoss()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Split the dataset into training and validation sets
indices = list(range(len(images)))
train_idx, val_idx = train_test_split(indices, test_size=0.2, random_state=42)

train_images = [images[i] for i in train_idx]
train_masks = [masks[i] for i in train_idx]
val_images = [images[i] for i in val_idx]
val_masks = [masks[i] for i in val_idx]

train_dataset = UltrasoundDataset(train_images, train_masks)
val_dataset = UltrasoundDataset(val_images, val_masks)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    epoch_loss = 0

    for images, masks in train_loader:
        images = images.to(device)
        masks = masks.to(device)

        outputs = model(images)

        loss = loss_fn(outputs, masks.float())

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_loader)}")

NameError: name 'torch' is not defined

In [ ]:
def iou_score(preds, targets, threshold=0.5):
    preds = (preds > threshold).float()
    targets = targets.float()
    
    intersection = (preds * targets).sum(dim=(1,2))  # sum over H and W
    union = preds.sum(dim=(1,2)) + targets.sum(dim=(1,2)) - intersection

    iou = (intersection + 1e-6) / (union + 1e-6)  # avoid divide by zero
    return iou.mean()

def dice_score(preds, targets, threshold=0.5):
    preds = (preds > threshold).float()
    targets = targets.float()
    
    intersection = (preds * targets).sum(dim=(1,2))  # sum over H and W
    dice = (2. * intersection + 1e-6) / (preds.sum(dim=(1,2)) + targets.sum(dim=(1,2)) + 1e-6)
    return dice.mean()

In [ ]:
model.eval()
with torch.no_grad():
    for images, masks in val_loader:
        images = images.to(device)
        masks = masks.to(device)

        outputs = model(images)
        outputs = outputs.squeeze(1)  # remove channel dim

        iou = iou_score(outputs, masks)
        dice = dice_score(outputs, masks)

        print(f"IoU: {iou.item():.4f}, Dice: {dice.item():.4f}")

TypeError: pad(): argument 'input' (position 1) must be Tensor, not numpy.ndarray